# Let's predict for the first chromosome
## Let's try to load data via DataLoader to predict (to avoid indexing error!)

In [1]:
# Import necessary packages
import numpy as np
import sys
import os
import pandas as pd
from Bio import SeqIO
from matplotlib import pyplot as plt
from sklearn.metrics import r2_score
from scipy.stats import pearsonr
import math
import torch
import torch.nn as nn
from tqdm.auto import tqdm
import re
import csv

from enformer_pytorch import Enformer, seq_indices_to_one_hot, GenomeIntervalDataset
from enformer_loader import GenomeDataIntervalDataset
import ast
from enformer_pytorch import from_pretrained
from enformer_pytorch.finetune import HeadAdapterWrapper

/software/hgi/installs/conda-audited/miniforge/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print('using GPU') if torch.cuda.is_available() else print('using CPU')

using GPU


In [2]:
# get file paths
path = 'tests/data/'
fasta_file = '/lustre/scratch126/gengen/projects/graft/Dataset/reference/hg38_galGal6_full/fasta/GRCh38.GRCg6a.full.renamed.merged.fa'
bed_file = path + 'test1_train_dataset.bed'
chroms = ['hg38_1']
print(f'File paths loaded \n')
# and load pre-trained model
weight_file = '/lustre/scratch126/gengen/teams/parts/jh47/enformer_pytorch/model_weights/'
if torch.cuda.is_available():
    print(f'Using GPU\n')
    enformer = from_pretrained(weight_file, use_checkpointing = True).cuda()
    print(f"Pre-trained model loaded.")
else:
    print('using CPU')


File paths loaded 

Using GPU

Pre-trained model loaded.


In [77]:
# let's try with our toy dataset - using dataloader it will not give us an error
from torch.utils.data import DataLoader
dataset = GenomeDataIntervalDataset(
    all_chroms=['hg38_1'],
    bed_file=bed_file,
    fasta_file=fasta_file)
data_loader = DataLoader(dataset, batch_size=4, shuffle=False)

with torch.no_grad():
    i = 0
    y_hat_vector = []
    for batch in data_loader:
        i += 1
        s = batch[0].cuda()
        y_hat = enformer(s, head = 'human')
        y_hat_vector.append(y_hat)

print(f"{i} batches were predicted")


/nfs/users/nfs_s/sb79/.local/lib/python3.12/site-packages/torch/utils/checkpoint.py:553: UserWarning: torch.utils.checkpoint.checkpoint_sequential: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/nfs/users/nfs_s/sb79/.local/lib/python3.12/site-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


10 batches were predicted


In [82]:
# get our desired track!
predtest = []
for i in range(len(y_hat_vector)):
    #batchpred = y_hat_vector[i]['human'].cpu().detach().squeeze(0).numpy()
    for j in range(y_hat_vector[0].shape[0]):
        predtest.append(y_hat_vector[i][j][:,1436].cpu().detach().numpy())

print(f"There are {len(predtest[0])} values in each of the {len(predtest)} predictions.")

There are 896 values in each of the 40 predictions.


# Let's simply predict for genomic windows of 196kb

### Get ranges

In [3]:
# jacob's code
seq_len = 196608
# Only center 114688 basepairs are used, 320 bins cropped on either side.
PRED_WIDTH = seq_len - (320 * 128) - (320 * 128)
OUT_N_BINS = (PRED_WIDTH // 128)
PRED_WIDTH

114688

In [90]:
# jacob's code (modified)
chrom_length = 248956422 # for hg38_1
intervals = [("hg38_1", s, s+seq_len) for s in range(0, chrom_length - seq_len, PRED_WIDTH) if s >0]

intervals

[('hg38_1', 114688, 311296),
 ('hg38_1', 229376, 425984),
 ('hg38_1', 344064, 540672),
 ('hg38_1', 458752, 655360),
 ('hg38_1', 573440, 770048),
 ('hg38_1', 688128, 884736),
 ('hg38_1', 802816, 999424),
 ('hg38_1', 917504, 1114112),
 ('hg38_1', 1032192, 1228800),
 ('hg38_1', 1146880, 1343488),
 ('hg38_1', 1261568, 1458176),
 ('hg38_1', 1376256, 1572864),
 ('hg38_1', 1490944, 1687552),
 ('hg38_1', 1605632, 1802240),
 ('hg38_1', 1720320, 1916928),
 ('hg38_1', 1835008, 2031616),
 ('hg38_1', 1949696, 2146304),
 ('hg38_1', 2064384, 2260992),
 ('hg38_1', 2179072, 2375680),
 ('hg38_1', 2293760, 2490368),
 ('hg38_1', 2408448, 2605056),
 ('hg38_1', 2523136, 2719744),
 ('hg38_1', 2637824, 2834432),
 ('hg38_1', 2752512, 2949120),
 ('hg38_1', 2867200, 3063808),
 ('hg38_1', 2981888, 3178496),
 ('hg38_1', 3096576, 3293184),
 ('hg38_1', 3211264, 3407872),
 ('hg38_1', 3325952, 3522560),
 ('hg38_1', 3440640, 3637248),
 ('hg38_1', 3555328, 3751936),
 ('hg38_1', 3670016, 3866624),
 ('hg38_1', 3784704, 39

In [85]:
int((seq_len - PRED_WIDTH) /2)

40960

In [75]:
chrom_length = 248956422 # for hg38_1
intervals = [("hg38_1", s - int((seq_len - PRED_WIDTH) /2), s+seq_len+int((seq_len - PRED_WIDTH) /2)) for s in range(0, chrom_length - seq_len, PRED_WIDTH) if s>0]
# we now have intervals
intervals

[('hg38_1', 73728, 352256),
 ('hg38_1', 188416, 466944),
 ('hg38_1', 303104, 581632),
 ('hg38_1', 417792, 696320),
 ('hg38_1', 532480, 811008),
 ('hg38_1', 647168, 925696),
 ('hg38_1', 761856, 1040384),
 ('hg38_1', 876544, 1155072),
 ('hg38_1', 991232, 1269760),
 ('hg38_1', 1105920, 1384448),
 ('hg38_1', 1220608, 1499136),
 ('hg38_1', 1335296, 1613824),
 ('hg38_1', 1449984, 1728512),
 ('hg38_1', 1564672, 1843200),
 ('hg38_1', 1679360, 1957888),
 ('hg38_1', 1794048, 2072576),
 ('hg38_1', 1908736, 2187264),
 ('hg38_1', 2023424, 2301952),
 ('hg38_1', 2138112, 2416640),
 ('hg38_1', 2252800, 2531328),
 ('hg38_1', 2367488, 2646016),
 ('hg38_1', 2482176, 2760704),
 ('hg38_1', 2596864, 2875392),
 ('hg38_1', 2711552, 2990080),
 ('hg38_1', 2826240, 3104768),
 ('hg38_1', 2940928, 3219456),
 ('hg38_1', 3055616, 3334144),
 ('hg38_1', 3170304, 3448832),
 ('hg38_1', 3284992, 3563520),
 ('hg38_1', 3399680, 3678208),
 ('hg38_1', 3514368, 3792896),
 ('hg38_1', 3629056, 3907584),
 ('hg38_1', 3743744, 402

In [86]:
466944 - 188416

278528

In [91]:
intervals[0][1] + ((320) * 128) # first actual result

155648

In [92]:
intervals[-1][2] - ((320) * 128) # last actual result

248913920

In [94]:
n_intervals = len(intervals) # the number of intervals
# now we want to store the bins, which are 128 bp long. We take the start of our actual result, and the end, and split it in 128bp bins.
all_pred_intervals = [(s, s+128) for s in range(intervals[0][1] + ((320) * 128), intervals[-1][2] - ((320) * 128), 128)]
len(all_pred_intervals) # we should have all of these values!

1943424

In [95]:
# let's write a .bed file to load later
with open("pred_hg38_1.bed", "w") as bed_file:
    # Loop through the list of tuples and write each line
    for region in intervals:
        chrom, start, end = region
        bed_file.write(f"{chrom}\t{start}\t{end}\n")

print("BED file has been written as 'pred_hg38_1.bed'")

BED file has been written as 'pred_hg38_1.bed'


### Now we have our ranges and the coordinates of our results. Let's predict. I'll use some of Jacob's code.

In [96]:
targets_txt = 'targets_human.txt' # change to targets_mouse.txt if mouse
chrom = 'hg38_1'
chrom_sizes_path = path + 'all_chrom_len.txt'
chrom_sizes = pd.read_csv(chrom_sizes_path, sep='\t', header=None, index_col=0)
chrom_sizes.columns = ['size']
chrom_length = chrom_sizes.loc[chrom].item()


In [46]:
pred_dir = path + f'/predictions/{chrom}'
if not os.path.exists(pred_dir):
    os.makedirs(pred_dir)

In [97]:
import pysam
import pyfaidx
fa_loc = '/lustre/scratch126/gengen/projects/graft/Dataset/reference/hg38_galGal6_full/fasta/GRCh38.GRCg6a.full.renamed.merged.fa'
pyfaidx.Faidx(fa_loc)
fasta_open = pysam.Fastafile(fa_loc)

In [98]:
# our targets
targets_predicted_list = {
        'HEK293_H3K4me3': 'CHIP:H3K4me3:HEK293',
        'HEK293_H3K9me3': 'CHIP:H3K9me3:HEK293'
        }

In [99]:
targets_df = pd.read_csv(targets_txt, sep='\t')
targets_to_pred = {}
for key, val in targets_predicted_list.items():
    targets_to_pred[key] = targets_df.loc[targets_df['description'] == val]['index'].tolist()

# Check if all targets contain at least one track
assert sum([len(targets_to_pred[key]) == 0 for key in targets_to_pred.keys()]) == 0, 'Some targets are not found'


In [100]:
targets_to_pred

{'HEK293_H3K4me3': [1176], 'HEK293_H3K9me3': [1436]}

In [101]:
# we can create a data loader with our bed and fasta file:
from enformer_pytorch import GenomeIntervalDataset
from torch.utils.data import DataLoader
bed_file = 'pred_hg38_1.bed'
datachr1 = GenomeIntervalDataset(
    bed_file=bed_file,
    fasta_file=fasta_file)
data_loader = DataLoader(datachr1, batch_size=1, shuffle=False)
# batch size of 1 to save computation time

In [102]:
len(data_loader)

2169

In [103]:
# pred loop
print(f'Batch 1/{len(data_loader)}')
with torch.no_grad():
    y_hat_vector = []
    for i, batch in enumerate(data_loader):
        if (i + 1) % 10 == 0:
            print(f'Batch {i+1}/{len(data_loader)}')
        s = batch.cuda()
        y_hat = enformer(s, head = 'human')
        y_hat_vector.append(y_hat)

Batch 1/2169


/nfs/users/nfs_s/sb79/.local/lib/python3.12/site-packages/torch/utils/checkpoint.py:553: UserWarning: torch.utils.checkpoint.checkpoint_sequential: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/nfs/users/nfs_s/sb79/.local/lib/python3.12/site-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Batch 10/2169
Batch 20/2169
Batch 30/2169
Batch 40/2169
Batch 50/2169
Batch 60/2169
Batch 70/2169
Batch 80/2169
Batch 90/2169
Batch 100/2169
Batch 110/2169
Batch 120/2169
Batch 130/2169
Batch 140/2169
Batch 150/2169
Batch 160/2169
Batch 170/2169
Batch 180/2169
Batch 190/2169
Batch 200/2169
Batch 210/2169
Batch 220/2169
Batch 230/2169
Batch 240/2169
Batch 250/2169
Batch 260/2169
Batch 270/2169
Batch 280/2169
Batch 290/2169
Batch 300/2169
Batch 310/2169
Batch 320/2169
Batch 330/2169
Batch 340/2169
Batch 350/2169
Batch 360/2169
Batch 370/2169
Batch 380/2169
Batch 390/2169
Batch 400/2169
Batch 410/2169
Batch 420/2169
Batch 430/2169
Batch 440/2169
Batch 450/2169
Batch 460/2169
Batch 470/2169
Batch 480/2169
Batch 490/2169
Batch 500/2169
Batch 510/2169
Batch 520/2169
Batch 530/2169
Batch 540/2169
Batch 550/2169
Batch 560/2169
Batch 570/2169
Batch 580/2169
Batch 590/2169
Batch 600/2169
Batch 610/2169
Batch 620/2169
Batch 630/2169
Batch 640/2169
Batch 650/2169
Batch 660/2169
Batch 670/2169
Batc

In [105]:
print(f"Shape of the prediction: {y_hat_vector[0].shape}. There are {len(y_hat_vector)} predictions.")
# y_hat_vector[0].shape

Shape of the prediction: torch.Size([1, 896, 5313]). There are 2169 predictions.


Great we have predicted things!

In [108]:
len(all_pred_intervals)
# all_pred_intervals

1943424

In [109]:
# get our desired track!
predk9 = []
for i in range(len(y_hat_vector)):
    #batchpred = y_hat_vector[i]['human'].cpu().detach().squeeze(0).numpy()
    for j in range(y_hat_vector[0].shape[0]):
        predk9.append(y_hat_vector[i][j][:,1436].cpu().detach().numpy())

print(f"There are {len(predk9[0])} values in each of the {len(predk9)} H3K9me3 predictions.")

predk4 = []
for i in range(len(y_hat_vector)):
    #batchpred = y_hat_vector[i]['human'].cpu().detach().squeeze(0).numpy()
    for j in range(y_hat_vector[0].shape[0]):
        predk4.append(y_hat_vector[i][j][:,1176].cpu().detach().numpy())

print(f"There are {len(predk4[0])} values in each of the {len(predk4)} H3K4me3 predictions.")


There are 896 values in each of the 2169 H3K9me3 predictions.
There are 896 values in each of the 2169 H3K4me3 predictions.


In [110]:
allpredk4 = []
allpredk9 = []
for i in range(len(predk4)):
    for j in range(len(predk4[0])):
        allpredk4.append(predk4[i][j])
        allpredk9.append(predk9[i][j])


In [111]:
# I use this to store the values:
all_target_preds = dict(zip(
        targets_to_pred.keys(),
        [np.zeros((len(all_pred_intervals), 1)) for i in range(len(targets_to_pred.keys()))]
    ))
print(f"The shape of target is  {all_target_preds['HEK293_H3K4me3'].shape}.")
len(all_target_preds['HEK293_H3K4me3'])

The shape of target is  (1943424, 1).


1943424

In [34]:
len(predk4)

2169

In [36]:
len(allpredk4)

1943424

In [67]:
len(all_pred_intervals)

1944064

In [58]:
all_pred_intervals

[(114688, 114816),
 (114816, 114944),
 (114944, 115072),
 (115072, 115200),
 (115200, 115328),
 (115328, 115456),
 (115456, 115584),
 (115584, 115712),
 (115712, 115840),
 (115840, 115968),
 (115968, 116096),
 (116096, 116224),
 (116224, 116352),
 (116352, 116480),
 (116480, 116608),
 (116608, 116736),
 (116736, 116864),
 (116864, 116992),
 (116992, 117120),
 (117120, 117248),
 (117248, 117376),
 (117376, 117504),
 (117504, 117632),
 (117632, 117760),
 (117760, 117888),
 (117888, 118016),
 (118016, 118144),
 (118144, 118272),
 (118272, 118400),
 (118400, 118528),
 (118528, 118656),
 (118656, 118784),
 (118784, 118912),
 (118912, 119040),
 (119040, 119168),
 (119168, 119296),
 (119296, 119424),
 (119424, 119552),
 (119552, 119680),
 (119680, 119808),
 (119808, 119936),
 (119936, 120064),
 (120064, 120192),
 (120192, 120320),
 (120320, 120448),
 (120448, 120576),
 (120576, 120704),
 (120704, 120832),
 (120832, 120960),
 (120960, 121088),
 (121088, 121216),
 (121216, 121344),
 (121344, 12

In [64]:
len(allpredk4)

1943424

In [112]:
with open(f'{pred_dir}/enformer_fullpreds_H3K4me3.bedGraph', 'w') as f:
    f.write(f'track type=bedGraph name="enformer predictions (H3K4me3)" description="enformer predictions (H3K4me3)"\n')
    for i, ((s, e), scorek4) in enumerate(zip((all_pred_intervals), allpredk4)):
        f.write(f'{chrom}\t{s}\t{e}\t{scorek4}\n')

In [113]:
with open(f'{pred_dir}/enformer_fullpreds_H3K9me3.bedGraph', 'w') as f:
    f.write(f'track type=bedGraph name="enformer predictions (H3K9me3)" description="enformer predictions (H3K9me3)"\n')
    for i, ((s, e), scorek9) in enumerate(zip((all_pred_intervals), allpredk9)):
        f.write(f'{chrom}\t{s}\t{e}\t{scorek9}\n')

## Try this with their Enformer model function? Note: this was really bad.

In [114]:
model = Enformer.from_hparams(
    dim = 1536,
    depth = 11,
    heads = 8,
    output_heads = dict(human = 5313, mouse = 1643)
).cuda()

In [115]:
# pred loop
print(f'Batch 1/{len(data_loader)}')
with torch.no_grad():
    y_hat_vector = []
    for i, batch in enumerate(data_loader):
        if (i + 1) % 10 == 0:
            print(f'Batch {i+1}/{len(data_loader)}')
        s = batch.cuda()
        y_hat = model(s, head = 'human')
        y_hat_vector.append(y_hat)

Batch 1/2169
Batch 10/2169
Batch 20/2169
Batch 30/2169
Batch 40/2169
Batch 50/2169
Batch 60/2169
Batch 70/2169
Batch 80/2169
Batch 90/2169
Batch 100/2169
Batch 110/2169
Batch 120/2169
Batch 130/2169
Batch 140/2169
Batch 150/2169
Batch 160/2169
Batch 170/2169
Batch 180/2169
Batch 190/2169
Batch 200/2169
Batch 210/2169
Batch 220/2169
Batch 230/2169
Batch 240/2169
Batch 250/2169
Batch 260/2169
Batch 270/2169
Batch 280/2169
Batch 290/2169
Batch 300/2169
Batch 310/2169
Batch 320/2169
Batch 330/2169
Batch 340/2169
Batch 350/2169
Batch 360/2169
Batch 370/2169
Batch 380/2169
Batch 390/2169
Batch 400/2169
Batch 410/2169
Batch 420/2169
Batch 430/2169
Batch 440/2169
Batch 450/2169
Batch 460/2169
Batch 470/2169
Batch 480/2169
Batch 490/2169
Batch 500/2169
Batch 510/2169
Batch 520/2169
Batch 530/2169
Batch 540/2169
Batch 550/2169
Batch 560/2169
Batch 570/2169
Batch 580/2169
Batch 590/2169
Batch 600/2169
Batch 610/2169
Batch 620/2169
Batch 630/2169
Batch 640/2169
Batch 650/2169
Batch 660/2169
Batch 

In [116]:
print(f"Shape of the prediction: {y_hat_vector[0].shape}. There are {len(y_hat_vector)} predictions.")
# y_hat_vector[0].shape

Shape of the prediction: torch.Size([1, 896, 5313]). There are 2169 predictions.


In [117]:
# get our desired track!
predk9 = []
for i in range(len(y_hat_vector)):
    #batchpred = y_hat_vector[i]['human'].cpu().detach().squeeze(0).numpy()
    for j in range(y_hat_vector[0].shape[0]):
        predk9.append(y_hat_vector[i][j][:,1436].cpu().detach().numpy())

print(f"There are {len(predk9[0])} values in each of the {len(predk9)} H3K9me3 predictions.")

predk4 = []
for i in range(len(y_hat_vector)):
    #batchpred = y_hat_vector[i]['human'].cpu().detach().squeeze(0).numpy()
    for j in range(y_hat_vector[0].shape[0]):
        predk4.append(y_hat_vector[i][j][:,1176].cpu().detach().numpy())

print(f"There are {len(predk4[0])} values in each of the {len(predk4)} H3K4me3 predictions.")


There are 896 values in each of the 2169 H3K9me3 predictions.
There are 896 values in each of the 2169 H3K4me3 predictions.


In [118]:
allpredk4 = []
allpredk9 = []
for i in range(len(predk4)):
    for j in range(len(predk4[0])):
        allpredk4.append(predk4[i][j])
        allpredk9.append(predk9[i][j])


In [119]:
with open(f'{pred_dir}/enformer_fullpreds_H3K4me3_model2.bedGraph', 'w') as f:
    f.write(f'track type=bedGraph name="enformer model2 predictions (H3K4me3)" description="enformer predictions (H3K4me3)"\n')
    for i, ((s, e), scorek4) in enumerate(zip((all_pred_intervals), allpredk4)):
        f.write(f'{chrom}\t{s}\t{e}\t{scorek4}\n')

In [120]:
with open(f'{pred_dir}/enformer_fullpreds_H3K9me3_model2.bedGraph', 'w') as f:
    f.write(f'track type=bedGraph name="enformer model2 predictions (H3K9me3)" description="enformer predictions (H3K9me3)"\n')
    for i, ((s, e), scorek9) in enumerate(zip((all_pred_intervals), allpredk9)):
        f.write(f'{chrom}\t{s}\t{e}\t{scorek9}\n')

## Now try to do it with jacob's helper functions... (?)